In [7]:
#from jira import JIRA
import json
import time
import math
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import inspect
import time
import random
import numpy as np
from string import punctuation

In [2]:
class Jira_API_Query():
    
    def __init__(self, server, email, psw):
        self.server = server
        self.email = email
        self.psw = psw
        
        self.options = {'server': self.server}
        self.jira = JIRA(self.options, basic_auth=(self.email, self.psw))
    
    def retrieve_tickets(self,query, num_tickets):
        
        tickets = []
        ceil = math.ceil(num_tickets/100)
        
        for i in range(ceil):
            hundred_tickets = self.jira.search_issues(query, startAt=(i*100) ,maxResults=100)
            for ticket in hundred_tickets:
                tickets.append(ticket)
            time.sleep(2)
            if ((i+1)*100)%1000==0:
                print("Loaded tickets from "+str((i-9)*100)+" to "+ str((i+1)*100))
        
        return tickets

### Here tickets are retrieved and loaded locally in lists

In [45]:
authenticated = Jira_API_Query(server = 'https://your/jira/url.com', email="your_email.com",\
                  psw = "your_password)

In [46]:
tickets = authenticated.retrieve_tickets('YOUR_JIRA_QUERY',13000)

Loaded tickets from 0 to 1000
Loaded tickets from 1000 to 2000
Loaded tickets from 2000 to 3000
Loaded tickets from 3000 to 4000
Loaded tickets from 4000 to 5000
Loaded tickets from 5000 to 6000
Loaded tickets from 6000 to 7000
Loaded tickets from 7000 to 8000
Loaded tickets from 8000 to 9000
Loaded tickets from 9000 to 10000
Loaded tickets from 10000 to 11000
Loaded tickets from 11000 to 12000
Loaded tickets from 12000 to 13000


In [89]:
tickets_saving = []
for ticket in tickets:
    id_ = ticket.id
    type_ = ticket.fields.issuetype.name
    labels = ticket.fields.labels
    if not ticket.fields.components:
        component = []
    else:
        component = ticket.fields.components[0].name
    summary = ticket.fields.summary
    description = ticket.fields.description
    priority = ticket.fields.priority.name
    project = ticket.fields.project.key
    
    tickets_saving.append([id_,type_,labels,component,priority,summary,description,project])

### Saving a list to a JSON object

In [93]:
#with open('JIRA.json', 'w') as f:
#    json.dump(tickets_saving, f) 

### Loading the local json Object

In [8]:
with open('JIRA.json', 'r') as f:
    jira = json.load(f)
df = pd.DataFrame(jira, columns=["ID","bug_cat","labels","components","priority","summary","description","project"])

### Here we consider filter only the tickets whose project is PROJECT_A

In [9]:
condition = df["project"]=="YOUR_PROJECT"
PROJECT_A = df[condition].iloc[:,:-1].reset_index().drop("index",axis=1)

### We save in a list the tickets whose description is empty and then we drop them

In [10]:
idx_weird = []
tot_chars = 0
for i in range(len(PROJECT_A)):
    try:
        a = len(PROJECT_A["description"][i])
    except:
        idx_weird.append(i)
    tot_chars += a
PROJECT_A = PROJECT_A.drop(index = idx_weird, axis=0).reset_index().drop("index",axis=1)

In [12]:
# COMPUTE AVG CHARS PER TICKET DESCRIPTION
tot_chars = 0
for i in range(len(PROJECT_A)):
    a = len(PROJECT_A["description"][i])
    tot_chars += a
print(tot_chars//PROJECT_A.shape[0])

556


### 1. preprocessing DESCRIPTION

### Cleaning the text

In [14]:
def remove_chunk_(text,start_char,end_char,middle_str=""):
    words = text.split()
    for i in range(len(text.split())):
        if start_char in words[i] and end_char in words[i]:
            words.remove(words[i])
            words.insert(i,middle_str)
    new_text = " ".join(words)
    return new_text

In [15]:
def clean(text):
    text = remove_chunk_(text,"http","html"," <LINK> ")
    text = remove_chunk_(text,"http","com"," <SITE> ")
    text = remove_chunk_(text,"@","."," <MAIL> ")
    text = remove_chunk_(text,'{color:','}', " <COLOR_OBJ> ")
    return text

In [16]:
def remove_thumbnails(text, middle_str=""):
    words = text.split()
    for i in range(len(words)):
        if words[i].endswith("thumbnail!"):
            words.remove(words[i])
            words.insert(i,middle_str)       
    descr = " ".join(words)
    return descr

#### The funtion below I shown only partially because it contained some private-data

In [17]:
def preprocess(descript, punctuation_=True):
     
    design = None
    # The "->" in a non ASCII character that needs to be treated in this way "'\\u2013>'"
    if "–>" in descript:
        descript = descript.partition("–>")[0]+"->" + descript.partition("—>")[2]
    # The "->" in a non ASCII character that needs to be treated in this way "'\\u2014>'"
    if "—>" in descript:
        descript = descript.partition("—>")[0]+"->" + descript.partition("—>")[2]
    
    arrows = ["-->","->","==>","=>"]
    descript = re.sub("|".join(arrows)," <ARROW> ",descript)
    semi_arrows = [">","<"]
    descript = re.sub("|".join(semi_arrows),"",descript)
    
    descript = descript.replace("\r\n\r\n", "")
    descript = descript.replace("\r\n", " <RETURN> ")
    
    #here removing images, links and email
    descript = remove_thumbnails(descript, " <IMAGE> ")
    descript = clean(descript)
    
    if punctuation_:
        descript = descript.replace(" <RETURN>  <RETURN> ", " <RETURN> ")
        descript = descript.replace("\r\n", " <RETURN> ")
        descript = descript.replace("#"," <HASHTAG> ")
        descript = descript.replace("..."," ")
        descript = descript.replace("."," <PERIOD> ")
        descript = descript.replace(","," <COMMA> ")
        descript = descript.replace('"'," <QUOTATION_MARK> ")
        descript = descript.replace(";"," <SEMICOLON> ")
        descript = descript.replace("!"," <EXCLAMATION_MARK> ")
        descript = descript.replace("?"," <QUESTION_MARK> ")
        descript = descript.replace("("," <LEFT_PAREN> ")
        descript = descript.replace(")"," <RIGHT_PAREN> ")
        descript = descript.replace("--"," <HYPENS> ")
        descript = descript.replace("?"," <QUESTION_MARK> ")
        descript = descript.replace(":"," <COLON> ")
        descript = descript.replace("*"," ")
        descript = descript.replace("<RETURN>","")
        descript = descript.replace("|", " ")
    else:
        descript = descript.replace("<ARROW>"," ")
        for char in punctuation:
            descript = descript.replace(char, " ")
        
        descript = descript.replace("\r\n", " <RETURN> ")
        descript = descript.replace("<RETURN>"," ")
        descript = descript.replace("<RETURN>"," ")
        
        
    descript = descript.lower()
    descript = descript.replace("{color}", " ")
    descript = descript.split()

    return descript

#### Preprocessing step 1: removing most of raw text and splitting

In [19]:
import time
start = time.time()
preprocessed = []
for i in range(len(PROJECT_A["description"])):
    ppticket = preprocess(PROJECT_A.loc[i,"description"],punctuation_=False)
    preprocessed.append(ppticket)
end = time.time()
print(end-start)

1.379516839981079


#### And here a second round of preprocessing

In [20]:
def preprocess_2_phase(list_,previous_word="",word_to_remove="",replace_word=None):
    assert isinstance(list_,list), " The input must be a list of lists"
    assert previous_word != "", "must be a word that links to word_to_remove"
    idx_to_remove = []
    if previous_word in list_:
        for i in range(len(list_)):
            if list_[i].startswith(word_to_remove):
                if previous_word in list_[i-1]:
                    idx_to_remove.append(i-1)
                    idx_to_remove.append(i)
    new_list = []
    for i in range(len(list_)):
        if i not in idx_to_remove:
            new_list.append(list_[i])
        if not replace_word:
            continue
        else:
             if i in idx_to_remove and i-1 not in idx_to_remove:
                new_list.append(replace_word)
    
    return new_list

#### preprocessing step-2 --> removing other "image-"s from the text

In [21]:
start = time.time()
preprocessed_2 = []
for i in range(len(preprocessed)):
    ppticket_2 = preprocess_2_phase(preprocessed[i],"<exclamation_mark>","image-"," <image> ")
    preprocessed_2.append(ppticket_2)
end = time.time()
print(end-start)

0.15140819549560547


In [23]:
#with open('NO_PUNCT/PP_TICKETS_NO_PUNCT.json', 'w') as f:
    # indent=2 is not needed but makes the file human-readable
#    json.dump(preprocessed_2, f) 

In [25]:
with open("NO_PUNCT/PP_TICKETS_NO_PUNCT.json", "r") as f:
    tickets = json.load(f)

In [26]:
# otherwise you fucked up
len(tickets) == len(PROJECT_A)

True

### 2. preprocessing SUMMARY 
#### previously I cleant the description of the tickets, now I am cleaning the introduction ("SUMMARY")

In [28]:
def summary_cleaner(text,chars_chunk,signs):
    if isinstance(signs,list):
        for sign in signs:
            first_chunk = text[:chars_chunk].split(sign[0])
            first_chunk = " ".join(first_chunk)
            text = first_chunk + text[chars_chunk:]
        return text
    else:
        first_chunk = text[:chars_chunk].split(signs)
        first_chunk = " ".join(first_chunk)
        text = first_chunk + text[chars_chunk:]
        return text

In [29]:
import time
start = time.time()
summaries_ = []
for i in range(len(PROJECT_A["summary"])):
    summ_ = PROJECT_A["summary"][i]
    summary = summary_cleaner(summ_,len(summ_),["[","]"])
    summary = summary.replace("_"," ")
    summary = summary.replace("/"," <slash> ")
    pp_summ = preprocess(summary,punctuation_=False)
    pp_summ.extend(["<summary>"])
    summaries_.append(pp_summ)
end = time.time()
print(end-start)

0.4088099002838135


In [31]:
#with open("NO_PUNCT/SUMMARIES_NO_PUNCT.json","w") as f:
#    json.dump(summaries_,f)

#### THE END